In [1]:
import numpy as np
import xarray as xr
import datetime as dt
import os
import matplotlib.pyplot as plt

In [2]:
# Input directory for NMME forecast output
dirin = 'https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME'
# Output directory for downloads
dirout = '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW'#/NMME_SST/by_lead
# Model names
mods = ['CanCM4i-IC3',]#'GEM5.2NEMO',]#'CanCM4i', 'COLA-RSMAS-CCSM4', 'GEM-NEMO','GFDL-SPEAR', 'NASA-GEOSS2S',
# Number of time steps to grab at each opendap call (to avoid download
# failures)
time_chunk = 10;

In [3]:
imod=0
modi=mods[imod]
modi

'CanCM4i-IC3'

In [5]:
fin=f'{dirin}/.NCDC-OISST/.sst/dods'

# options: 'hindcast' or 'forecast': handle models with separate forecast directories
# Can models use a different order of dimensions from others (managed by xarray) 
print(f'\nDownloading OISST (start time {dt.datetime.now()})\n')
print('Progress\n--------')
print(fin)
fi=xr.open_dataset(fin,decode_times=False)



Progress
--------
https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.NCDC-OISST/.sst/dods


In [6]:
fi

<xarray.Dataset> Size: 91MB
Dimensions:  (T: 348, Y: 181, X: 360)
Coordinates:
  * T        (T) float32 1kB 264.5 265.5 266.5 267.5 ... 608.5 609.5 610.5 611.5
  * Y        (Y) float32 724B 90.0 89.0 88.0 87.0 ... -87.0 -88.0 -89.0 -90.0
  * X        (X) float32 1kB 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
Data variables:
    sst      (T, Y, X) float32 91MB ...
Attributes:
    Conventions:  IRIDL

In [7]:
fi.T

<xarray.DataArray 'T' (T: 348)> Size: 1kB
array([264.5, 265.5, 266.5, ..., 609.5, 610.5, 611.5], dtype=float32)
Coordinates:
  * T        (T) float32 1kB 264.5 265.5 266.5 267.5 ... 608.5 609.5 610.5 611.5
Attributes:
    standard_name:  time
    pointwidth:     1.0
    calendar:       360
    gridtype:       0
    units:          months since 1960-01-01

In [11]:
lon = fi.X.values # Lon
lat = fi.Y.values # Lat
time = fi.T.values # start time (months)
nt = len(time)
sst=np.empty((nt,len(lat),len(lon)))
fout = f'{dirout}/OISST/sst_NMME_1deg.monthly_1982_2010.nc'
# Subset times
tstart = 0
for tt in range(0,np.ceil(nt/time_chunk).astype(int)):
    tcount = min(time_chunk,nt-tstart+1)
    isdone = False
    while not isdone: # Loop to ensure download succeeds
        try: 
            sst[tstart:tstart+tcount,:,:]=fi.sst.isel(T=slice(tstart,(tstart+tcount)),).values
            tstart = tstart + tcount
            isdone = True
        except:
            raise
            print(' x')                

xout=xr.Dataset(data_vars={'lon':(['X',],lon),
                        'lat':(['Y',],lat),
                        'time':(['T'],time),
                        'sst':(['T','Y','X'],sst)},
            coords=dict(X=fi.X,Y=fi.Y,T=fi.T),
            attrs={'description':'downloaded from http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME'})
xout.to_netcdf(fout,mode='w')
del sst; del xout; fi.close()
print('done')

done


In [12]:
fi=xr.open_dataset(fin,decode_times=False)

In [14]:
fi.T

<xarray.DataArray 'T' (T: 348)> Size: 1kB
array([264.5, 265.5, 266.5, ..., 609.5, 610.5, 611.5], dtype=float32)
Coordinates:
  * T        (T) float32 1kB 264.5 265.5 266.5 267.5 ... 608.5 609.5 610.5 611.5
Attributes:
    standard_name:  time
    pointwidth:     1.0
    calendar:       360
    gridtype:       0
    units:          months since 1960-01-01

In [16]:
print(fi.T.attrs)
print(fi.T[0].values/12+1960,fi.T[-1].values/12+1960)
fi.close()

{'standard_name': 'time', 'pointwidth': 1.0, 'calendar': '360', 'gridtype': 0, 'units': 'months since 1960-01-01'}
1982.0416666666667 2010.9583333333333
